# GraphRAG

## Import packages

In [29]:
import os
import nltk
import string
import numpy as np
import pandas as pd
from unidecode import unidecode
from sklearn.metrics.pairwise import cosine_similarity
from tqdm import tqdm
from pathlib import Path
from rouge_score import rouge_scorer
import json
from dotenv import load_dotenv
from getpass import getpass

from langchain_community.embeddings import OllamaEmbeddings
from langchain.embeddings.cache import CacheBackedEmbeddings
from langchain.storage import LocalFileStore

from neurorag.neurorag import NeuroRAG

## Disable warnings

In [30]:
import warnings
warnings.filterwarnings('ignore')

## Setup environment variables

You have to define the following environment variables in the `.env` file, terminal environment, or input field within this Jupyter notebook:
1. MISTRAL_API_KEY
2. OPENAI_API_KEY
3. OPENAI_PROXY
4. TAVILY_API_KEY
5. ENTREZ_EMAIL

## Import packages

In [31]:
env_variables = [
  'MISTRAL_API_KEY',
  'OPENAI_API_KEY',
  'OPENAI_PROXY',
  'TAVILY_API_KEY',
  'ENTREZ_EMAIL',
]

load_dotenv()

for key in env_variables:
  value = os.getenv(key)

  if value is None:
    value = getpass(key)

  os.environ[key] = value

## Setup metrics

### Download NLTK dictionaries

These dictionaries are needed for further text preprocessing.

In [32]:
dict_ids = [
  'punkt_tab',
  'punkt',
  'stopwords',
  'wordnet',
]

for dict_id in dict_ids:
  nltk.download(dict_id, quiet=True)

### Text preprocessing

Define a function for text preprocessing, which is an important step before calculating any metrics. This preprocessing function will help in cleaning the text data, making it ready for further analysis. The preprocessing involves several steps:
1. Lowercasing
2. Stopwords removal
3. Lemmatization
4. Remove accents from characters

In [33]:
lemmatizer = nltk.stem.WordNetLemmatizer()

def preprocess(corpus: str) -> str:
  corpus = corpus.lower()
  stopset = nltk.corpus.stopwords.words('english') + nltk.corpus.stopwords.words('russian') + list(string.punctuation)
  tokens = nltk.word_tokenize(corpus)
  tokens = [t for t in tokens if t not in stopset]
  tokens = [lemmatizer.lemmatize(t) for t in tokens]
  corpus = ' '.join(tokens)
  corpus = unidecode(corpus)
  return corpus

### Embedding Initialization

Here we are initializing the Llama 3 embeddings model. The `OllamaEmbeddings` class is a component of the Ollama library, a set of pre-trained language models. This model is capable of embedding corpora of any length into a 4096-dimensional vector.

The use of `OllamaEmbeddings` requires the installation of a local Ollama server, which can be found at https://ollama.com.

In [34]:
embeddings = OllamaEmbeddings(model='llama3.1')
store = LocalFileStore("./.embeddings_cache")

cached_embeddings = CacheBackedEmbeddings.from_bytes_store(
  embeddings,
  store,
  namespace=embeddings.model,
)

### Average embeddings cosine similarity metric

This function calculates the average cosine similarity between expected answers and LLM predicted answers using their respective embeddings. Cosine similarity is a measure of similarity between two non-zero vectors of an inner product space that measures the cosine of the angle between them:

$$
K(a, b) = \frac{\sum \limits_{i=1}^n a_i b_i}{\sqrt{\sum \limits_{i=1}^n a_i^2} \cdot \sqrt{\sum \limits_{i=1}^n b_i^2}}
$$

In [35]:
def embeddings_cosine_sim_metric(expected_answers: list[str], predicted_answers: list[str]) -> float:
  results = []

  for expected_answer, predicted_answer in zip(expected_answers, predicted_answers):
    expected_answer = preprocess(expected_answer)
    predicted_answer = preprocess(predicted_answer)

    expected_embedding = np.array(cached_embeddings.embed_query(expected_answer))
    predicted_embedding = np.array(cached_embeddings.embed_query(predicted_answer))

    sim = cosine_similarity(
      expected_embedding.reshape(1, -1),
      predicted_embedding.reshape(1, -1),
    )[0][0]

    results.append(sim)

  return np.mean(results)

In [36]:
smoothie_f = nltk.translate.bleu_score.SmoothingFunction().method4

def bleu_metric(expected_answers, predicted_answers):
  scores = []

  for expected_answer, predicted_answer in zip(expected_answers, predicted_answers):
    expected_answer = preprocess(expected_answer)
    predicted_answer = preprocess(predicted_answer)

    predicted_tokens = nltk.word_tokenize(predicted_answer)
    expected_tokens = [nltk.word_tokenize(expected_answer)]

    score = nltk.translate.bleu_score.sentence_bleu(
      expected_tokens,
      predicted_tokens,
      smoothing_function=smoothie_f,
    )

    scores.append(score)

  return np.mean(scores)

In [37]:
rogue_1_scorer = rouge_scorer.RougeScorer(['rouge1'], use_stemmer=True)

def rogue_1_metric(expected_answers, predicted_answers):
  scores = []

  for expected_answer, predicted_answer in zip(expected_answers, predicted_answers):
    expected_answer = preprocess(expected_answer)
    predicted_answer = preprocess(predicted_answer)

    result = rogue_1_scorer.score(expected_answer, predicted_answer)

    scores.append(result['rouge1'])

  return np.mean(scores)

In [38]:
rogue_l_scorer = rouge_scorer.RougeScorer(['rougeL'], use_stemmer=True)

def rogue_l_metric(expected_answers, predicted_answers):
  scores = []

  for expected_answer, predicted_answer in zip(expected_answers, predicted_answers):
    expected_answer = preprocess(expected_answer)
    predicted_answer = preprocess(predicted_answer)

    result = rogue_l_scorer.score(expected_answer, predicted_answer)

    scores.append(result['rougeL'])

  return np.mean(scores)

## Build model

In [87]:
app = NeuroRAG()
app.compile()

In [79]:
app.invoke({'question': 'Count each amino acid in the ABW05875 sequence'})

---DETERMINE SPECIALIZED SOURCES---
---ROUTE QUESTION---
---ROUTE QUESTION TO SPECIALIZED SOURCES: NCBI_PROTEIN---
---GENERATE STEP-BACK QUERY---
---GENERATE REWRITTEN QUERY---
---GENERATE SUBQUERIES---
---FINAL SUBQUERIES NUMBER: 3---
---GENERATE HYDE DOCUMENTS---
---RETRIEVE FROM NCBI PROTEIN DB---
---CHECK DOCUMENT RELEVANCE TO QUESTION---
---INITIAL DOCUMENTS NUMBER: 3---
---GRADE DOCUMENT (1/1)---
---GRADE: DOCUMENT NOT RELEVANT---
---FINAL DOCUMENTS NUMBER: 0---
---ASSESS GRADED DOCUMENTS---
---DECISION: SOME DOCUMENTS ARE NOT RELEVANT TO QUESTION, INCLUDE WEB SEARCH---
---WEB SEARCH---
---GENERATE---


Fusing candidates: 100%|██████████| 1/1 [00:00<00:00,  1.69it/s]


---CHECK HALLUCINATIONS---
---DECISION: GENERATION IS NOT GROUNDED IN DOCUMENTS, RE-TRY---
---GENERATE---


Fusing candidates: 100%|██████████| 1/1 [00:00<00:00,  1.78it/s]

---CHECK HALLUCINATIONS---


{'question': 'Count each amino acid in the ABW05875 sequence',
 'specialized_srcs': ['ncbi_protein'],
 'step_back_query': 'What are the general characteristics of protein sequences like ABW05875?',
 'rewritten_query': 'Count each amino acid in the ABW05875 sequence',
 'subqueries': ['What are the counts of each amino acid type (A, C, D, E, F, G, H, I, K, L, M, N, P, Q, R, S, T, V, W, Y) in the ABW05875 sequence?',
  'How many occurrences of each amino acid are there in the ABW05875 sequence?',
  'What is the frequency distribution of each amino acid type in the ABW05875 sequence?'],
 'generated_docs': ['Here is a scientific paper-style passage answering the question:\n\n**Title:** Amino Acid Composition Analysis of the ABW05875 Sequence\n\n**Abstract:**\nThe ABW05875 protein sequence, a member of the hypothetical protein family, has been analyzed to determine its amino acid composition. This study aimed to count and quantify each amino acid present in the ABW05875 sequence.\n\n**Result

## Evaluate RAG

### Load QA dataset

In [88]:
qa_df = pd.read_csv('brainscape.csv')[:500]
qa_df

,question,answer
0,What are the afferent cranial nerve nuclei?,Trigeminal sensory nucleus- fibres carry gener...
1,What is the order of the cranial nerves ?,1-olfactory\n2-optic\n3-oculomotor\n4-trochlea...
2,What are the efferent cranial nerve nuclei?,Edinger-westphal nucleus\nOculomotor nucleus\n...
3,Which nuclei share the embryo logical origin -...,Oculomotor nucleus Trochlear nucleus Abducens ...
4,Which nuclei share the embryo logical origin- ...,Trigeminal motor nucleus Facial motor nucleus ...
...,...,...
495,What are the functions each of these structure...,Cortex is involved in perception Hypothalamus ...
496,What are the stimuli for olfaction ?,Airborne molecules \nAlcohols \nEsters \nAroma...
497,What percentage of human genome are involved w...,3%
498,What is the difference between threshold value...,Lipid soluble have low thresholds while water ...


### Load cached RAGs responses

In [89]:
cache_path = Path('cache.json')

if not os.path.exists(cache_path):
  data = {}
  with open(cache_path, 'w') as file:
    json.dump(data, file)

with open(cache_path, 'r') as f:
  cache = json.load(f)

len(cache.keys())

282

In [93]:
questions = list(qa_df['question'].tolist())
expected_answers = list(qa_df['answer'].tolist())
predicted_answers = []

for index, question in tqdm(enumerate(questions)):
  if not question in cache:
    cache[question] = app.invoke({'question': question})['generation']

  predicted_answers.append(cache[question])

  with open(cache_path, 'w') as f:
    json.dump(cache, f)

cos_score = embeddings_cosine_sim_metric(expected_answers, predicted_answers)
bleu_score = bleu_metric(expected_answers, predicted_answers)
rogue_1_score = rogue_1_metric(expected_answers, predicted_answers)
rogue_l_score = rogue_l_metric(expected_answers, predicted_answers)

cos_score, bleu_score, rogue_1_score, rogue_l_score

361it [00:00, 1787.28it/s]

---DETERMINE SPECIALIZED SOURCES---
---ROUTE QUESTION---
---ROUTE QUESTION TO SPECIALIZED SOURCES: VECTORSTORE---
---GENERATE STEP-BACK QUERY---
---GENERATE REWRITTEN QUERY---
---GENERATE SUBQUERIES---
---FINAL SUBQUERIES NUMBER: 4---
---GENERATE HYDE DOCUMENTS---
---RETRIEVE FROM VECTOR STORE---
---CHECK DOCUMENT RELEVANCE TO QUESTION---
---INITIAL DOCUMENTS NUMBER: 28---
---GRADE DOCUMENT (1/4)---
---GRADE: DOCUMENT NOT RELEVANT---
---GRADE DOCUMENT (2/4)---
---GRADE: DOCUMENT NOT RELEVANT---
---GRADE DOCUMENT (3/4)---
---GRADE: DOCUMENT NOT RELEVANT---
---GRADE DOCUMENT (4/4)---
---GRADE: DOCUMENT NOT RELEVANT---
---FINAL DOCUMENTS NUMBER: 0---
---ASSESS GRADED DOCUMENTS---
---DECISION: SOME DOCUMENTS ARE NOT RELEVANT TO QUESTION, INCLUDE WEB SEARCH---
---WEB SEARCH---
---GENERATE---


Fusing candidates: 100%|██████████| 1/1 [00:00<00:00,  1.00it/s]


---CHECK HALLUCINATIONS---
---DECISION: GENERATION IS GROUNDED IN DOCUMENTS---
---GRADE GENERATION vs QUESTION---


470it [01:31,  3.56it/s]  

...

Fusing candidates: 100%|██████████| 1/1 [00:01<00:00,  1.43s/it]


---CHECK HALLUCINATIONS---
---DECISION: GENERATION IS GROUNDED IN DOCUMENTS---
---GRADE GENERATION vs QUESTION---


500it [1:13:17,  8.79s/it] 

---DECISION: GENERATION ADDRESSES QUESTION---


(0.6251679069211469,
 0.025192447599560224,
 0.2227855995692924,
 0.19491690991905436)